In [1]:
import numpy as np
import cv2
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import os
import random
import mediapipe as mp
import threading

In [2]:
# Initialize empty lists for data and labels
data = []
labels = []

# Define the path to your dataset directory
dataset_dir = "Image_Data"

# Initialize hand tracking
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize a lock
lock = threading.Lock()

In [3]:
# Function to extract hand landmarks from a frame
def extract_hand_landmarks(frame):
    with lock:
        results = hands.process(frame)
    if results.multi_hand_landmarks:
        landmarks = results.multi_hand_landmarks[0].landmark  # Assuming one hand is present
        return np.array([(landmark.x, landmark.y, landmark.z) for landmark in landmarks]).flatten()
    else:
        return None

In [4]:
# Iterate through each subdirectory (class) in the dataset directory
for sub_dir in os.listdir(dataset_dir):
    class_label = sub_dir
    sub_dir_path = os.path.join(dataset_dir, sub_dir)
    
    if os.path.isdir(sub_dir_path):  # Check if it's a directory
        # List all image files in the subdirectory
        image_files = os.listdir(sub_dir_path)
        
        # Randomly select 30 images from each class
        num_images_to_select = min(120, len(image_files))
        selected_images = random.sample(image_files, num_images_to_select)

        
        # Iterate through the selected images
        for image_file in selected_images:
            image_path = os.path.join(sub_dir_path, image_file)
            image = cv2.imread(image_path)  # Read the image
            
            # Resize the image to 320x240 pixels
            image = cv2.resize(image, (320, 240))
            
            # Extract hand landmarks as features
            landmarks = extract_hand_landmarks(image)
            
            if landmarks is not None:
                data.append(landmarks)
                labels.append(class_label)

In [5]:
# Convert lists to NumPy arrays
data = np.array(data)
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize the SVM classifier
svm_classifier = svm.SVC(kernel='linear')

# Train the SVM model on the training data
svm_classifier.fit(X_train, y_train)

SVC(kernel='linear')

In [6]:
# Initialize webcam
cap = cv2.VideoCapture(0)

# Set frame size to 320x240
cap.set(3, 320)
cap.set(4, 240)

current_gesture = ""  # Variable to store the current recognized gesture

In [7]:
# Function to update the gesture recognition result
def update_gesture(frame, scaler, svm_classifier):
    global current_gesture
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    
    # Resize the grayscale frame to 320x240 pixels
    resized = cv2.resize(gray, (320, 240))
    
    # Extract hand landmarks as features
    landmarks = extract_hand_landmarks(frame)
    
    if landmarks is not None:
        flattened = landmarks.flatten()
        flattened = scaler.transform([flattened])
        prediction = svm_classifier.predict(flattened)
        
        # Lock to ensure thread safety while updating current_gesture
        with lock:
            current_gesture = prediction[0]

In [8]:
# Function to continuously update gesture recognition result
def gesture_update_thread():
    while True:
        ret, frame = cap.read()  # Capture a frame from the webcam
        
        if not ret:
            break  # Break the loop if frame capture fails
        
        update_gesture(frame, scaler, svm_classifier)

In [9]:
# Start the gesture recognition thread
gesture_thread = threading.Thread(target=gesture_update_thread)
gesture_thread.daemon = True
gesture_thread.start()

while True:
    ret, frame = cap.read()  # Capture a frame from the webcam
    
    if not ret:
        break  # Break the loop if frame capture fails
    
    # Display the recognized gesture on the frame
    cv2.putText(frame, f"Gesture: {current_gesture}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display the frame
    cv2.imshow("Webcam Gesture Recognition", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the OpenCV windows
cap.release()
cv2.destroyAllWindows()